In [1]:
import warnings 
warnings.filterwarnings("ignore")

In [2]:
%run ../analysis_utils/lang_comparer.py
%run ../analysis_utils/chunk_comparer.py

#### Utils

In [34]:
def merge_results(results, merge_on="chunk_bucket"):
    model_names = list(results.keys())
    model_dfs = list(results.values())
    merged = model_dfs[0]
    for i, (model_name, model_df) in enumerate(zip(model_names[1:], model_dfs[1:])):
        if merge_on == "chunk_bucket":
            model_df = model_df.drop(columns=["chunk_buckets", "count"])
        merged = merged.merge(model_df, on=merge_on, suffixes=("", f"_{model_name}"))
    return merged

### Lang Comparison

In [46]:
baseline_lang_comparer = LangComparer(model="baseline")
baseline_df = baseline_lang_comparer.run()

chunk_lang_comparer = LangComparer(model="chunk_combinations")
chunk_df = chunk_lang_comparer.run()

results = {
    "baseline": baseline_df,
    "chunk_combinations": chunk_df
}

merged_df = merge_results(results, merge_on="lang_pair")

In [47]:
merged_df.head()

,count,mean_corr,count_chunk_combinations,mean_corr_chunk_combinations
lang_pair,,,,
de_de,498,0.6825,498,0.666667
es_en,381,0.7025,381,0.666667
it_it,351,0.7250,351,0.693333
en_en,264,0.7125,264,0.663333
es_it,251,0.6500,251,0.483333


### Chunk Comparison

In [32]:
baseline_chunk_comparer = ChunkComparer(model="baseline")
baseline_chunk_df = baseline_chunk_comparer.run()

chunk_chunk_comparer = ChunkComparer(model="chunk_combinations")
chunk_chunk_df = chunk_chunk_comparer.run()

results = {
    "baseline": baseline_chunk_df,
    "chunk_combinations": chunk_chunk_df
}

In [43]:
merged_df = merge_results(results, merge_on="chunk_bucket")
merged_df["corr_diff_percentage"] = round((merged_df["avg_corr_chunk_combinations"] - merged_df["avg_corr"]) / merged_df["avg_corr"] * 100, 2)

merged_df

,count,chunk_buckets,avg_corr,avg_corr_chunk_combinations,corr_diff_percentage
chunk_bucket,,,,,
2,1341,"(0, 2)",0.6725,0.593333,-11.77
4,1241,"(2, 4)",0.6900,0.643333,-6.76
6,702,"(4, 6)",0.6800,0.636667,-6.37
10,377,"(8, 10)",0.7125,0.650000,-8.77
12,312,"(10, 12)",0.7175,0.703333,-1.97
8,235,"(6, 8)",0.6450,0.633333,-1.81
16,192,"(14, 16)",0.6725,0.636667,-5.33
20,78,"(18, 20)",0.6650,0.600000,-9.77
18,58,"(16, 18)",0.6400,0.640000,0.00
